# Descarga de imágenes de Sentinel-2 desde Google Earth Engine

**[Google Earth Engine](https://earthengine.google.com) (GEE)** es una plataforma de Cloud Computing para procesar imágenes de satélite y otros datos geoespaciales. Posee un inmenso catálogo de productos satelitales y el poder computacional para procesarlos. Esta plataforma utiliza una interfaz de programación en JavaScript para cargar datos, ejecutar algoritmos y visualizarlos para programar la ejecución de las acciones que se realizarán por parte del servidor de GEE.

GEE también posee un API de [Python](https://www.python.org/download/releases/3.0/) para poder ejecutar las órdenes de servidor desde un computador personal. La ventaja del uso de este lenguaje por sobe JavaScript es su similutud con R tanto en el uso de librerías como en la sintaxis de uso.

Para conocer más de uso del API de GEE en Python referirse a los siguientes enlances:
- [Instalación del paquete del API](https://developers.google.com/earth-engine/guides/python_install)
- [Primeros pasos de GEE en Python](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)

## Uso del script de descarga de escenas de Sentinel-2

Al igual que en **R**, el primer paso que se debe realizar es cargar las librerías que contienen las funciones a utilizar en el script. Estas deben ser instaladas previamente, que dependiendo del sistema operativo usado. Por ejemplo, para MacOS y Linux, la instalación es mediante el comando `pip install earthengine-api` en Terminal o Shell-Script.

Las librerías utilizadas en este script son:
- `ee`: Librería que contiene el API oficial de GEE
- `geetools`: Librería de Rodrigo *Fito* Principe, programador argentino experto en recursos naturales y GEE. Revisar en: [gee_tools en GitHub](ehttps://github.com/gee-community/gee_tools)

Con sólo estas dos librerías se puede realizar todo el procedimiento de descarga. Al incluir la forma `from geetools import ui, cloud_mask` se están cargando ciertos módulos para que las funciones estén en el espacio de trabajo y no tener que utilizar la forma `nombre_libreria.nombre_funcion()`.

In [1]:
import ee
import geetools
from geetools import ui, cloud_mask

Luego de cargar las librerías, se debe autorizar el uso del API mediante una cuenta Google. Para ello, se debe ejecutar la función `ee.Authenticate()`, la que abrirá una ventana emergente para conceder el acceso desde una cuenta Google. Luego de autorizar, aparecerá un *token* o código de autorización que se debe ingresar en el cuadro `Enter verification code:`.

Si se autoriza correctamente, aparecerá el mensaje `Successfully saved authorization token.` indicando que el token se ha guardado y es posible utilizar el API.

In [2]:
ee.Authenticate()

Enter verification code:  4/1AX4XfWgUzdEd_VDq8dP0JwJZgBToWyBs_PuhtkTigJpld7oUdf9Gxd6QcYo



Successfully saved authorization token.


Para iniciar el API se debe ejecutar la siguiente función:

In [3]:
ee.Initialize()

Luego, se debe cargar la librería de las imágenes de Sentinel-2 en reflectancia de superficie (*Surface reflectance* o *SR* por sus siglas en inglés). Los datos técnicos de este producto están descritos en la [ficha técnica de Sentinel-2 nivel 2-A](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR). Este producto ya se encuentra corregido por un algoritmo de corrección atmosférica absoluta (Sen2Cor), a diferencia del [producto 1-C](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2) que se distribuye sen reflectancia al tope de la atmósfera (*Top of Atmosphere* o *TOA* por sus siglas en inglés).

En cada una de las fichas técnicas de ambos productos podrán revisar las bandas que incluyen así como las propiedades de las imágenes.

In [4]:
sentinel = ee.ImageCollection("COPERNICUS/S2_SR")

El siguiente paso es definir el área del polígono a cortar mediante las coordenadas de los vértices. Para este caso, se está mostrando el ejemplo para el grupo 1, indicando las coordenadas del rectángulo delimitador del área de estudio.

Se están definiendo 3 argumentos en la función, `coords`, `proj` y `geodesic`. El primero indica los pares de coordenadas para la definición del rectángulo delimitador; el segundo, el Sistena de Referencia de Coordenadas en código EPSG; el terceno, si los bordes son geodésicos siguiendo la curvatura de la tierra entre un vértice y otro, que en este caso está desactivado para obtener bordes planos entre par de coordenadas y otro.

Para obtener ayuda en la definición de los argumentos del las funciones de Python, utilizar la función `help`, como: `help(ee.Geometry.Polygon)`.

Para obtener fácilmente las coordenadas de los vértices del polígono, se recomienda el uso de QGIS para la creación del polígono y a extensión [GeoJSON](https://geojson.org) para la exportación. Basta con abrir el archivo resultante en un editor de texto para extraer fácilmente las coordenadas en el formato que GEE solicita.

In [5]:
# Polígono del grupo 1
poligono = ee.Geometry.Polygon(
        coords = [ [ [ -71.374531323682916, -30.732915541805554 ],
                    [ -71.372355526813735, -30.644023970318468 ],
                    [ -71.248312954038397, -30.646229555362691 ],
                    [ -71.250375226189533, -30.735128896921253 ],
                    [ -71.374531323682916, -30.732915541805554 ] ] ] , proj = 'EPSG:4326', geodesic = False);

A continuación, se debe definir la fecha de comienzo y término de la secuencia multi-temporal en formato 'año-mes-día':

In [7]:
# Fechas
SrtDate = '2020-07-01'
EndDate = '2021-11-01'

Una vez definiendo estos parámetros, se procede a cargar una función fundamental para facilitar el trabajo. La función `cloud_mask` enmascara las nubes en la escena de acuerdo a la banda de calidad de S2_SR (QA60). Luego se extrae la geometría de `poligono` para cortar las imágenes.

In [8]:
# desde Fito Principe
sentinel2function = cloud_mask.sentinel2()

# Geometría para realizar consulta y corte
area_corte = ee.Feature(poligono).geometry()

Finalmente, se procesa la colección de imágenes Sentinel-2. Cada función se va aplicando sobre el objeto `sentinel` que representa la colección. Las funciones son:
 - `filterBounds`: para filtrar las gratículas por límites del polígono.
 - `filterDate`: para filtrar por fecha.
 - `map`: para iterar la función de enmascarado de nubes en la colección.
 - `select`: para seleccionar las bandas que se desean exportar.
 
Como se observa, se van concatenando las funciones mediante el uso de puntos (`.`), el proceso se va ejecutando de izquierda a derecha. Por lo tanto lo primero es filtrar por los límites y lo último es la selección de las capas.

Primero se filtran las imágenes para reducir la colección y luego se aplican las funciones. En este caso, no se seleccionaron las bandas antes del emascarado de las nubes ya que la banda 'QA60' es necesaria para ello.
Otro enfoque podría haber sido seleccionar 'B2','B3','B4','B8' y 'QA60', ejecutar el proceso de enmascarado de nubes y luego descartar 'QA60'.

In [9]:
collection = sentinel.filterBounds(area_corte).filterDate(SrtDate,EndDate).map(sentinel2function).select(['B2','B3','B4','B8'])

Para salvar los resultados se utiliza la función `geetools.batch.Export.imagecollection.toDrive`. Esta guarda las imágenes en una carpeta de Drive en nuestra cuenta Google.

In [ ]:
print(collection)

In [11]:
geetools.batch.Export.imagecollection.toDrive(
            collection=collection,
            folder='S2_clasificacion',
            region=poligono,
            scale=10,
            crs= 'EPSG:32719',
            maxPixels=1e13
        )

[<Task AYPILWASENGQGPUYQQ6P7GZY EXPORT_IMAGE: 20200702T143731_20200702T144503_T19JBF (UNSUBMITTED)>,
 <Task ILKLIJYQYSNLJYITRC2DUSKL EXPORT_IMAGE: 20200702T143731_20200702T144503_T19JBG (UNSUBMITTED)>,
 <Task 6WJOE4JOUO3T75FY2MIG56SO EXPORT_IMAGE: 20200705T144731_20200705T145501_T19JBF (UNSUBMITTED)>,
 <Task DQM4SHDYOSKSXI2HJ5ANBFGU EXPORT_IMAGE: 20200705T144731_20200705T145501_T19JBG (UNSUBMITTED)>,
 <Task UMAFIK2ZPE4ASJFNJPTCKW7F EXPORT_IMAGE: 20200707T143729_20200707T144647_T19JBF (UNSUBMITTED)>,
 <Task O2W5QSLEW5NTFRXOWDA55IHA EXPORT_IMAGE: 20200707T143729_20200707T144647_T19JBG (UNSUBMITTED)>,
 <Task VIO63N2JYLQ72U4KBYJOY2B2 EXPORT_IMAGE: 20200710T144729_20200710T145606_T19JBF (UNSUBMITTED)>,
 <Task XD3VOSPB3ZWDDPBJALXPDZOQ EXPORT_IMAGE: 20200710T144729_20200710T145606_T19JBG (UNSUBMITTED)>,
 <Task EN6D2NTFNMLGUBY6LJAZVY6F EXPORT_IMAGE: 20200712T143731_20200712T143912_T19JBF (UNSUBMITTED)>,
 <Task STO5ASW6HK7FAAX3BOJWA3IH EXPORT_IMAGE: 20200712T143731_20200712T143912_T19JBG (UNSUB